In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import KFold
from sklearn import linear_model
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import collections, numpy
#import SMOTE_2 as smote

C:\Users\suchi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [207]:

data = pd.read_excel("quote_data_working.xlsx", sheetname="Sheet1")

data.count()
data.head()

,QuoteNumber,QuoteStatus,QuoteDay,QuoteDate,QuoteHour,QuoteTimeofDay,QuoteTime,prior_insurance_carrier_cd,PriorLimits,insurance_score,...,FollowUpIndicatorPriority,ADPF Indicator Group,DVS Group Group,DVS Report Status Gruop,IID Status Message Group,IID Hit Color,IID Report Status,Risk Group Name Group,Verification Path Name Group,Verification Path Reason
0,Q7304945907,Draft,Friday,"Friday, January 12, 2018",1 PM,Afternoon,13:36:53,NaN,NaN,NaN,...,2,No,Low Risk,Success,Record(s) Found,Yellow,Success,Group A,Manual + Verification,d. IID Risk Group: A
1,Q3999407819,Draft,Friday,"Friday, January 12, 2018",1 PM,Afternoon,13:38:12,GEICO ADVANTAGE INS,15000/30000,NaN,...,3,Yes,Low Risk,Success,Record(s) Found,Green,Success,Group C,Automated,NaN
2,Q5081290309,Draft,Saturday,"Saturday, January 13, 2018",7 PM,Evening,19:33:30,CSAA MID-ATLANTIC IC,100000/300000,NaN,...,1,Yes,Low Risk,Success,Record(s) Found,Green,Success,Group C,Automated,NaN
3,Q2793042841,Draft,Sunday,"Sunday, January 14, 2018",6 PM,Evening,18:52:37,OMNI INSURANCE COMPA,15000/30000,NaN,...,3,Yes,Low Risk,Success,Record(s) Found,Green,Success,Group C,Automated,NaN
4,Q8593473978,Draft,Monday,"Monday, January 15, 2018",8 AM,Morning,08:28:26,NaN,NaN,NaN,...,2,No,Low Risk,Success,Record(s) Found,Yellow,Success,Group A,Manual + Verification,d. IID Risk Group: A


In [208]:
#print(len(data['prior_insurance_carrier_cd'].unique())#.count())
data['prior_insurance_carrier_cd'].unique()
prior_ins=data['prior_insurance_carrier_cd']
#type(prior_ins)
#type(data['prior_insurance_carrier_cd'])

In [209]:
prior_ins_definition=['GEICO','ERIE', 'STATE FARM', 'PROGRESSIVE','AAA','AGENCY','ALLSTATE','ALLIED','AMICA','CSAA',
                     'FARMERS','HARTFORD','GRANGE','HRTFRD','MAPFRE','LIBERTY','LM','MET','NATIONWIDE','TRAVELERS','TRAVELLERS','USAA']

In [210]:
len(prior_ins)
type(prior_ins)
pi=list(prior_ins)

In [211]:
for n in range(len(pi)):
    i=str(pi[n]).upper()
    j=([sub for sub in prior_ins_definition if sub in i],i)
    if(j[0])==[]:
        pi[n]=j[1]
        #print(type(i))
    else:
        pi[n]=j[0][0]
    #print(i)
    
        #for sub in prior_ins_definition:
     #   if sub in i:
        

    #j=[i for sub in prior_ins_definition if sub not in i]#, i]
    #print(j)
    #else:
        #print("false")
    #print(any(substring in string for substring in substring_list))

In [212]:
newPi = pd.Series(pi)
data['new_prior_ins'] = newPi.values
#se = pd.Series(mylist)

In [213]:

print(data['new_prior_ins'].head()) # changed common insurance names and grouped to one kind


0                     NAN
1                   GEICO
2                    CSAA
3    OMNI INSURANCE COMPA
4                     NAN
Name: new_prior_ins, dtype: object


In [214]:
data.head()
tempdata=data

In [202]:
def flatten(ins_name, ins_list):
    new_list=[]
    for i in range(len(ins_list)):
        #print(ins_list[i])
        if(str(ins_list[i]) == str(ins_name)):
            new_list.append(1)
            #print("here")
        else:
            #new_list[i]=0
            new_list.append(0)
    #print(ins_list)
    return new_list


In [229]:
#prior_ins_definition=['GEICO','ERIE', 'STATE FARM', 'PROGRESSIVE','AAA','AGENCY','ALLSTATE','ALLIED','AMICA','CSAA'
#                     'FARMERS','HARTFORD','GRANGE','HRTFRD','MAPFRE','LIBERTY','LM','MET','NATIONWIDE','TRAVELERS','TRAVELLERS',
#                      'USAA']


data['GEICO'], data['PROGRESSIVE'] = [flatten('GEICO',list(data['new_prior_ins'])),flatten('PROGRESSIVE',list(data['new_prior_ins']))]
data['ERIE'], data['STATE FARM'] = [flatten('ERIE',list(data['new_prior_ins'])),flatten('STATE FARM',list(data['new_prior_ins']))]
data['AAA'], data['AGENCY'] = [flatten('AAA',list(data['new_prior_ins'])),flatten('AGENCY',list(data['new_prior_ins']))]
data['ALLSTATE'], data['ALLIED'] = [flatten('ALLSTATE',list(data['new_prior_ins'])),flatten('ALLIED',list(data['new_prior_ins']))]
data['AMICA'], data['CSAA'] = [flatten('AMICA',list(data['new_prior_ins'])),flatten('CSAA',list(data['new_prior_ins']))]
data['FARMERS'], data['HARTFORD'] = [flatten('FARMERS',list(data['new_prior_ins'])),flatten('HARTFORD',list(data['new_prior_ins']))]
data['GRANGE'], data['HRTFRD'] = [flatten('GRANGE',list(data['new_prior_ins'])),flatten('HRTFRD',list(data['new_prior_ins']))]
data['MAPFRE'], data['LIBERTY'] = [flatten('MAPFRE',list(data['new_prior_ins'])),flatten('LIBERTY',list(data['new_prior_ins']))]
data['LM'], data['MET'] = [flatten('LM',list(data['new_prior_ins'])),flatten('MET',list(data['new_prior_ins']))]
data['NATIONWIDE'], data['TRAVELERS'] = [flatten('NATIONWIDE',list(data['new_prior_ins'])),flatten('TRAVELERS',list(data['new_prior_ins']))]
data['USAA'], data['NAN'] = [flatten('USAA',list(data['new_prior_ins'])),flatten('NAN',list(data['new_prior_ins']))]

In [236]:
data.head()

,QuoteNumber,QuoteStatus,QuoteDay,QuoteDate,QuoteHour,QuoteTimeofDay,QuoteTime,prior_insurance_carrier_cd,PriorLimits,insurance_score,...,GRANGE,HRTFRD,MAPFRE,LIBERTY,LM,MET,NATIONWIDE,TRAVELERS,USAA,NAN
0,Q7304945907,Draft,Friday,"Friday, January 12, 2018",1 PM,Afternoon,13:36:53,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
1,Q3999407819,Draft,Friday,"Friday, January 12, 2018",1 PM,Afternoon,13:38:12,GEICO ADVANTAGE INS,15000/30000,NaN,...,0,0,0,0,0,0,0,0,0,0
2,Q5081290309,Draft,Saturday,"Saturday, January 13, 2018",7 PM,Evening,19:33:30,CSAA MID-ATLANTIC IC,100000/300000,NaN,...,0,0,0,0,0,0,0,0,0,0
3,Q2793042841,Draft,Sunday,"Sunday, January 14, 2018",6 PM,Evening,18:52:37,OMNI INSURANCE COMPA,15000/30000,NaN,...,0,0,0,0,0,0,0,0,0,0
4,Q8593473978,Draft,Monday,"Monday, January 15, 2018",8 AM,Morning,08:28:26,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1


In [227]:
#for i in range(len(prior_ins_definition)):
#    print(prior_ins_definition[i])
#    data[prior_ins_definition[i]]=[flatten(prior_ins_definition[i],list(data['new_prior_ins']))]

In [226]:
#pins=prior_ins_definition[2]
#pins=str('\''+pins+'\'')
#data[pins]=[flatten(pins,list(data['new_prior_ins']))]


In [203]:
#nn=flatten(ins_name,aa)
#nn
#if i in nn:
 #   if i==1:
  #      print(i)

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [237]:
#data.head()

In [201]:
aa=list(data['new_prior_ins'])
aa[1]=='GEICO'
ins_name='GEICO'

In [178]:
#data = pd.DataFrame({
 #   'GEICO': [flatten('GEICO',list(data['new_prior_ins']))],
  #  'PROGRESSIVE': [flatten('PROGRESSIVE',data['new_prior_ins']))]
#})

C:\Users\suchi\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\suchi\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
